# Transformações e exploração de dados por aparelho

## Converte todos os bancos de dados de DB para JSON

Este código Python tem como objetivo converter todos os bancos de dados SQLite em arquivos JSON. Ele permite uma manipulação mais flexível e análise dos dados coletados pelos smartphones.

### Funcionamento do código

1. **Lista de arquivos SQLite**: O código começa definindo uma lista `arquivos_sqlite` que contém os nomes dos arquivos SQLite que serão convertidos em arquivos JSON. Cada arquivo representa os dados coletados por um smartphone em um determinado período de tempo.

2. **Função `extrair_e_salvar_dados`**: Esta função é responsável por iterar sobre cada arquivo SQLite na lista `arquivos_sqlite`, extrair os dados de cada tabela presente no banco de dados e salvar esses dados em um arquivo JSON correspondente. Para cada tabela, os dados são organizados em um formato JSON e escritos em um arquivo JSON separado.

3. **Iteração sobre os arquivos SQLite**: O código itera sobre cada arquivo SQLite na lista `arquivos_sqlite` e aplica a função `extrair_e_salvar_dados` a cada um deles. Isso garante que todos os bancos de dados sejam convertidos em arquivos JSON de forma automatizada.

## Exploração de dados das bases de dados coletadas por smartphone

Após a conversão dos bancos de dados para arquivos JSON, é possível realizar diversas análises e explorações dos dados coletados pelos smartphones. Algumas possíveis explorações incluem:

- Análise da qualidade dos dados GNSS (latitude, longitude, altitude, precisão, etc.) ao longo do tempo e em diferentes locais.
- Estudo do comportamento de movimento do smartphone com base nos dados de velocidade e direção do GNSS.
- Identificação de padrões de uso do smartphone com base nos dados de timestamps e outras informações disponíveis nos bancos de dados.

Essas são apenas algumas das muitas possibilidades de exploração de dados que podem ser realizadas após a conversão dos bancos de dados para arquivos JSON. A análise e exploração dependem dos objetivos específicos do projeto e das questões de pesquisa a serem respondidas.

Se precisar de mais alguma explicação ou tiver outras dúvidas, estou à disposição para ajudar!


In [5]:
import sqlite3
import json

# Lista de nomes dos arquivos SQLite
arquivos_sqlite = ["Aparelho1Dia1.db", "Aparelho2Dia1.db", "Aparelho3Dia1.db", "Aparelho4Dia1.db",
                   "Aparelho1Dia2.db", "Aparelho2Dia2.db", "Aparelho3Dia2.db", "Aparelho4Dia2.db"]

# Função para extrair e salvar dados de um banco de dados
def extrair_e_salvar_dados(nome_banco):
    conexao = sqlite3.connect(nome_banco)
    cursor = conexao.cursor()

    # Lista para armazenar os dados convertidos
    dados_json = []

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tabelas = cursor.fetchall()
    for tabela in tabelas:
        nome_tabela = tabela[0]
        cursor.execute("SELECT * FROM {}".format(nome_tabela))
        dados_tabela = cursor.fetchall()

        # Crie um dicionário com os dados da tabela e adicione à lista
        for linha in dados_tabela:
            dicionario_linha = dict(zip([coluna[0] for coluna in cursor.description], linha))
            dados_json.append({nome_tabela: dicionario_linha})

    # Salve os dados em um arquivo JSON
    nome_arquivo_json = f"{nome_banco.replace('.db', '.json')}"
    with open(nome_arquivo_json, "w") as arquivo_json:
        json.dump(dados_json, arquivo_json, indent=4)

    # Feche a conexão com o banco de dados SQLite
    conexao.close()

# Iterar sobre os arquivos SQLite
for arquivo in arquivos_sqlite:
    extrair_e_salvar_dados(arquivo)


## Limpeza de estrutura json herdada do banco de dados

Este código Python tem como objetivo realizar uma limpeza na estrutura de dados de arquivos JSON que foram criados a partir de um banco de dados. Ele foca na organização do campo 'gnss_data_json', removendo a formatação adicional e mantendo apenas a lista de satélites.

### Funcionamento do código

1. **Lista de arquivos JSON**: O código começa definindo uma lista `arquivos_json` que contém os nomes dos arquivos JSON que serão processados. Esses arquivos são assumidos como sendo gerados a partir de um banco de dados, e contêm informações sobre os dados do GNSS (Global Navigation Satellite System).

2. **Função `organizar_gnss_data_json`**: Esta função é responsável por realizar a limpeza dos dados em cada arquivo JSON. Ela abre o arquivo JSON especificado, carrega os dados presentes nele e itera sobre cada elemento. Para cada elemento, verifica se há a chave 'gnss_data'. Se essa chave estiver presente e se o valor correspondente for uma string (em vez de um objeto JSON), a função converte essa string em um objeto JSON e atualiza o campo 'gnss_data_json' com a lista de satélites contida nesse objeto.

3. **Iteração sobre os arquivos JSON**: O código itera sobre cada arquivo JSON na lista `arquivos_json` e aplica a função `organizar_gnss_data_json` a cada um deles. Isso garante que todos os arquivos sejam processados e que a limpeza seja aplicada consistentemente.

Ao final da execução deste código, os arquivos JSON terão sido atualizados para conter apenas a lista de satélites no campo 'gnss_data_json', removendo a formatação adicional herdada do banco de dados e tornando os dados mais legíveis e fáceis de usar para análises posteriores.


In [7]:
import json
import os

# Lista de nomes dos arquivos JSON
arquivos_json = ["Aparelho1Dia1.json", "Aparelho2Dia1.json", "Aparelho3Dia1.json", "Aparelho4Dia1.json",
                 "Aparelho1Dia2.json", "Aparelho2Dia2.json", "Aparelho3Dia2.json", "Aparelho4Dia2.json"]

# Função para organizar o campo 'gnss_data_json' nos arquivos JSON
def organizar_gnss_data_json(arquivo_json):
    with open(arquivo_json, "r") as arquivo:
        dados = json.load(arquivo)
    
    for dado in dados:
        if 'gnss_data' in dado:
            if 'gnss_data_json' in dado['gnss_data'] and isinstance(dado['gnss_data']['gnss_data_json'], str):
                gnss_data_json_objeto = json.loads(dado['gnss_data']['gnss_data_json'])
                dado['gnss_data']['gnss_data_json'] = gnss_data_json_objeto.get('satellites', [])  # Organizar para conter apenas a lista de satélites
    
    with open(arquivo_json, "w") as arquivo:
        json.dump(dados, arquivo, indent=4)

# Iterar sobre os arquivos JSON
for arquivo_json in arquivos_json:
    organizar_gnss_data_json(arquivo_json)


## Normalização de Timestamp em Arquivos JSON

Este código Python tem como objetivo normalizar os timestamps presentes nos arquivos JSON contendo dados coletados por dispositivos móveis. A normalização do timestamp é feita convertendo o tempo absoluto para um valor normalizado dentro do intervalo de 0 a 1, representando os minutos desde o início do dia.

### Funcionamento do código

1. **Função `converter_timestamp`**: Esta função recebe um timestamp em milissegundos e o converte para um valor normalizado dentro do intervalo de 0 a 1, representando os minutos desde o início do dia. Isso é útil para comparar timestamps independentemente da data exata em que ocorreram.

2. **Lista de arquivos JSON**: O código define uma lista `arquivos_json` contendo os nomes dos arquivos JSON que serão processados. Esses arquivos contêm os dados coletados pelos dispositivos móveis e possuem timestamps que precisam ser normalizados.

3. **Iteração sobre os arquivos JSON**: O código itera sobre cada arquivo JSON na lista `arquivos_json`. Para cada arquivo, ele carrega os dados JSON, aplica a normalização de timestamp em cada registro e, em seguida, sobrescreve o arquivo original com os dados modificados.

## Exploração de dados normalizados

Após a normalização dos timestamps nos arquivos JSON, os dados podem ser explorados de forma mais consistente e eficiente. A normalização permite comparar eventos independentemente da data exata em que ocorreram, facilitando análises de padrões temporais e correlações entre diferentes variáveis.

Por exemplo, agora é possível comparar ocorrências de eventos em diferentes arquivos JSON com base na hora do dia em que ocorreram, independentemente da data. Isso pode ser útil para identificar padrões de comportamento que se repetem diariamente ou para analisar variações sazonais nos dados coletados.


In [8]:
import json
import datetime
import os

def converter_timestamp(timestamp):
    # Converter o timestamp para um objeto datetime
    data_hora = datetime.datetime.fromtimestamp(timestamp / 1000)  # Dividido por 1000 para converter de milissegundos para segundos
    # Calcular o número de minutos desde o início do dia
    minutos_do_dia = data_hora.hour * 60 + data_hora.minute
    # Normalizar os minutos do dia para o intervalo de 0 a 1
    minutos_normalizados = minutos_do_dia / (24 * 60)  # 24 horas * 60 minutos
    return minutos_normalizados

# Lista de nomes dos arquivos JSON
arquivos_json = ["Aparelho1Dia1.json", "Aparelho2Dia1.json", "Aparelho3Dia1.json", "Aparelho4Dia1.json",
                 "Aparelho1Dia2.json", "Aparelho2Dia2.json", "Aparelho3Dia2.json", "Aparelho4Dia2.json"]

# Iterar sobre os arquivos JSON
for arquivo_json in arquivos_json:
    # Carregar os dados do arquivo JSON
    with open(arquivo_json, "r") as arquivo:
        dados_json = json.load(arquivo)
    
    # Converter os timestamps em cada arquivo JSON
    for dado in dados_json:
        if 'gnss_data' in dado:
            gnss_data = dado['gnss_data']
            if 'timestamp' in gnss_data:
                gnss_data['timestamp'] = converter_timestamp(gnss_data['timestamp'])
    
    # Salvar os dados modificados no mesmo arquivo JSON
    with open(arquivo_json, "w") as arquivo:
        json.dump(dados_json, arquivo, indent=4)


## Cálculo da Diferença em Metros entre Coordenadas Coletadas e Reais

Este código Python tem como objetivo calcular a diferença em metros entre as coordenadas coletadas por dispositivos móveis e as coordenadas reais. Essa diferença é calculada em relação às coordenadas convertidas para o sistema UTM (Universal Transverse Mercator) para uma comparação precisa.

### Funcionamento do código

1. **Lista de arquivos JSON**: O código define uma lista `arquivos_json` contendo os nomes dos arquivos JSON que serão processados. Cada arquivo contém dados coletados por dispositivos móveis em diferentes aparelhos e dias.

2. **Iteração sobre os arquivos JSON**: O código itera sobre cada arquivo JSON na lista `arquivos_json`. Para cada arquivo, ele carrega os dados JSON, calcula a diferença em metros entre as coordenadas coletadas e reais e atualiza os valores no dicionário de dados.

3. **Cálculo da Diferença em Metros**: Para cada registro de dados, o código converte as coordenadas coletadas e reais para o sistema UTM. Em seguida, calcula a diferença em metros entre as coordenadas coletadas e reais e atualiza os valores no dicionário de dados.

4. **Salvando os dados ajustados**: Após calcular a diferença em metros para todos os registros de dados, o código salva os dados ajustados no mesmo arquivo JSON, sobrescrevendo os dados originais.

Este código é útil para análises que exigem uma comparação precisa entre as coordenadas coletadas por dispositivos móveis e as coordenadas reais. Por exemplo, pode ser utilizado para avaliar a precisão do GPS em diferentes condições e ambientes.


In [9]:
import json
import utm

# Função para converter strings para float
def converter_para_float(valor):
    try:
        return float(valor)
    except ValueError:
        return None

# Lista de arquivos JSON
arquivos_json = ["Aparelho1Dia1.json", "Aparelho2Dia1.json", "Aparelho3Dia1.json", "Aparelho4Dia1.json",
                 "Aparelho1Dia2.json", "Aparelho2Dia2.json", "Aparelho3Dia2.json", "Aparelho4Dia2.json"]

# Iterar sobre os arquivos JSON
for arquivo_json in arquivos_json:
    # Abrir o arquivo JSON original
    with open(arquivo_json, "r") as arquivo:
        dados = json.load(arquivo)

    # Iterar sobre os dados
    for dado in dados:
        if 'gnss_data' in dado:
            gnss_data = dado['gnss_data']
            # Converter coordenadas coletadas para UTM
            latitude_coletada = converter_para_float(gnss_data['latitude'])
            longitude_coletada = converter_para_float(gnss_data['longitude'])
            if latitude_coletada is not None and longitude_coletada is not None:
                latitude_utm_coletada, longitude_utm_coletada, _, _ = utm.from_latlon(latitude_coletada, longitude_coletada)
            
                # Converter coordenadas reais para UTM
                latitudereal = converter_para_float(gnss_data['latitudereal'])
                longitudereal = converter_para_float(gnss_data['longitudereal'])
                if latitudereal is not None and longitudereal is not None:
                    latitude_utm_real, longitude_utm_real, _, _ = utm.from_latlon(latitudereal, longitudereal)
            
                    # Calcular a diferença em metros
                    latitude_diferenca_m = latitude_utm_coletada - latitude_utm_real
                    longitude_diferenca_m = longitude_utm_coletada - longitude_utm_real
            
                    # Atualizar os valores no dicionário
                    gnss_data['latitudeDiferencaMetros'] = latitude_diferenca_m
                    gnss_data['longitudeDiferencaMetros'] = longitude_diferenca_m

    # Salvar os dados ajustados em um novo arquivo JSON
    with open(arquivo_json, "w") as arquivo_json_diferenca:
        json.dump(dados, arquivo_json_diferenca, indent=4)


## Limpeza do Campo 'gnss_data_json' em Arquivos JSON

Este código Python tem como objetivo limpar o campo 'gnss_data_json' em um arquivo JSON, removendo a estrutura externa e mantendo apenas a lista de satélites contida dentro dela. O campo 'gnss_data_json' normalmente contém informações sobre satélites coletadas por dispositivos móveis.

### Funcionamento do código

1. **Função `limpar_gnss_data_json`**: Esta função recebe uma lista de dicionários representando os dados JSON. Ela itera sobre cada dicionário e verifica se contém a chave 'gnss_data'. Se essa chave estiver presente e o valor associado for um dicionário, a função verifica se o campo 'gnss_data_json' está presente e é uma string. Se for o caso, converte a string em um objeto JSON e atualiza o campo 'gnss_data_json' com a lista de satélites. Em seguida, remove o campo 'gnss_data_json' do dicionário.

2. **Iteração sobre os arquivos JSON**: O código define uma lista `arquivos_json` contendo os nomes dos arquivos JSON que serão processados. Em seguida, itera sobre cada arquivo JSON na lista, carrega os dados, aplica a limpeza do campo 'gnss_data_json' e salva os dados limpos em um novo arquivo JSON.

Este código é útil para preparar os dados coletados por dispositivos móveis para análises subsequentes, removendo estruturas desnecessárias e mantendo apenas as informações relevantes sobre os satélites.

In [10]:
import json

# Função para limpar o campo 'gnss_data_json'
def limpar_gnss_data_json(dados):
    for dado in dados:
        if 'gnss_data' in dado:
            gnss_data = dado['gnss_data']
            # Verificar se 'gnss_data_json' está presente e é uma string
            if 'gnss_data_json' in gnss_data and isinstance(gnss_data['gnss_data_json'], str):
                # Converter a string em um objeto JSON
                gnss_data_json_objeto = json.loads(gnss_data['gnss_data_json'])
                # Atualizar o campo 'gnss_data_json' com a lista de satélites
                gnss_data['satellites'] = gnss_data_json_objeto.get('satellites', [])
                # Remover o campo 'gnss_data_json'
                del gnss_data['gnss_data_json']

# Lista de arquivos JSON
arquivos_json = ["Aparelho1Dia1.json", "Aparelho2Dia1.json", "Aparelho3Dia1.json", "Aparelho4Dia1.json",
                 "Aparelho1Dia2.json", "Aparelho2Dia2.json", "Aparelho3Dia2.json", "Aparelho4Dia2.json"]

# Iterar sobre os arquivos JSON
for arquivo_json in arquivos_json:
    # Abrir o arquivo JSON e carregar os dados
    with open(arquivo_json, "r") as arquivo:
        dados = json.load(arquivo)

    # Limpar o campo 'gnss_data_json'
    limpar_gnss_data_json(dados)

    # Sobrescrever o arquivo JSON com os dados limpos
    with open(arquivo_json, "w") as arquivo_json_limpo:
        json.dump(dados, arquivo_json_limpo, indent=4)


## Tratamento de tipagem de campos e alteração em estrutura

Neste script Python, realizaremos o processamento de dados JSON para aprimorar a estrutura e a qualidade dos dados. O objetivo é converter strings em floats para os campos 'latitudereal' e 'longitudereal', além de renomear e mover o campo 'gnss_data_json' para 'satelites'.

### Passos do Processamento:

1. **Importação de Bibliotecas:** Importaremos as bibliotecas necessárias para o processamento de dados.

2. **Definição das Funções:**
   - `converter_para_float`: Função para converter strings em floats para os campos 'latitudereal' e 'longitudereal'.
   - `renomear_e_mover`: Função para renomear e mover o campo 'gnss_data_json' para 'satelites'.
   - `processar_arquivo`: Função para processar um arquivo JSON, aplicando as funções de conversão e renomeação.

3. **Iteração sobre os Arquivos JSON:**
   - Iteraremos sobre os arquivos JSON especificados.
   - Para cada arquivo, realizaremos o processamento chamando a função `processar_arquivo`.


In [15]:
import json
import os

# Função para converter strings em floats para os campos 'latitudereal' e 'longitudereal'
def converter_para_float(dados):
    for dado in dados:
        if 'latitudereal' in dado:
            dado['latitudereal'] = float(dado['latitudereal'])
        if 'longitudereal' in dado:
            dado['longitudereal'] = float(dado['longitudereal'])

# Função para renomear e mover o campo 'gnss_data_json' para 'satelites'
def renomear_e_mover(dados):
    for dado in dados:
        if 'gnss_data_json' in dado:
            gnss_data = dado.pop('gnss_data_json')
            dado['satelites'] = gnss_data

# Função para processar um arquivo JSON
def processar_arquivo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        dados = json.load(arquivo)
        
    converter_para_float(dados)
    renomear_e_mover(dados)
    
    with open(caminho_arquivo, 'w') as arquivo:
        json.dump(dados, arquivo, indent=4)

# Diretório onde estão os arquivos JSON
diretorio_json = os.getcwd()  # Obtém o diretório atual de trabalho

# Lista de arquivos JSON
arquivos_json = ["Aparelho1Dia1.json", "Aparelho2Dia1.json", "Aparelho3Dia1.json", "Aparelho4Dia1.json",
                 "Aparelho1Dia2.json", "Aparelho2Dia2.json", "Aparelho3Dia2.json", "Aparelho4Dia2.json"]

# Iterar sobre os arquivos JSON
for arquivo_json in arquivos_json:
    caminho_arquivo = os.path.join(diretorio_json, arquivo_json)
    processar_arquivo(caminho_arquivo)

